# 작업형 제 2유형

- 데이터 전처리
- Feature Engineering
- 모델링(분류 알고리즘 사용)
- 하이퍼 파라미터 튜닝
- 모형 앙상블
- csv 제출

## 라이브러리 및 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
X = pd.read_csv("X_train.csv",encoding = 'cp949')
y = pd.read_csv("y_train.csv",encoding = 'cp949')
test = pd.read_csv("X_test.csv",encoding = 'cp949')

## 간단한 EDA

In [3]:
X.shape,y.shape, test.shape

((3500, 10), (3500, 2), (2482, 10))

In [4]:
X.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [5]:
y.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


In [6]:
X.isna().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [7]:
X.describe(include = "object")

,주구매상품,주구매지점
count,3500,3500
unique,42,24
top,기타,본 점
freq,595,1077


In [8]:
y['gender'].value_counts()

0    2184
1    1316
Name: gender, dtype: int64

## 데이터 전처리

In [9]:
X = X.fillna(0)
test = test.fillna(0)

In [10]:
X = X.drop(['cust_id'], axis = 1)
cust_id = test.pop('cust_id')

## Feature Engineering

In [11]:
# !pip install sklearn

In [12]:
from sklearn.preprocessing import LabelEncoder

In [21]:
cols = ['주구매상품','주구매지점']
for col in cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    test[col] = le.fit_transform(test[col])
    
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1
2,3197000,1639000,0.0,6,1,2,2.000000,0.000000,1
3,16077620,4935000,0.0,5,2,18,2.444444,0.318182,16
4,29050000,24000000,0.0,15,8,2,1.500000,0.000000,85


## 모델링 & 하이퍼 파라미터 튜닝

In [22]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 2022)
model.fit(X, y['gender'])
print(model.score(X, y['gender']))
predictions = model.predict_proba(test)

0.6874285714285714


In [23]:
pd.DataFrame(predictions)

,0,1
0,0.564328,0.435672
1,0.802744,0.197256
2,0.822674,0.177326
3,0.579338,0.420662
4,0.514488,0.485512
...,...,...
2477,0.525568,0.474432
2478,0.488952,0.511048
2479,0.562968,0.437032
2480,0.639971,0.360029


In [16]:
predictions[:,1]

array([0.43567157, 0.19725558, 0.17732635, ..., 0.43703219, 0.36002886,
       0.54383742])

In [17]:
output = pd.DataFrame({'cust_id':cust_id, 'gender':predictions[:,1]})

In [18]:
output.head()

,cust_id,gender
0,3500,0.435672
1,3501,0.197256
2,3502,0.177326
3,3503,0.420662
4,3504,0.485512
